In [1]:
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
from utility.others import clear_all
import time
from functions import NewWhaleGenerator,create_newWhale_model
from utility.grayscale_DenseNet import DenseNet_greyscale
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
#from tensorflow.keras.applications import DenseNet121

In [2]:
color = False
shapes = (224,224,1)
HalfBatch = 8

### set up generator

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [4]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1),Cutout(p=1)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

gen_train = NewWhaleGenerator(Ids_train.Imgs.tolist(),newWhale_train,transform,HalfBatch=HalfBatch)
gen_val = NewWhaleGenerator(Ids_val.Imgs.tolist(),newWhale_val,transform_test,HalfBatch=HalfBatch)

### build model

In [5]:
feature_model = load_model('Models/feature_model_DenseNet_Triplet_BN_MLP.h5')

In [6]:
conv_base = feature_model.layers[0]

In [7]:
conv_base.trainable = False

In [8]:
train_model = create_newWhale_model(1e-3,conv_base,nodes=[1024,1])

### training

In [9]:
# conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
# train_model,feature_model = create_model2(1e-3,margin_loss_fun_factory(0.1,5),conv_base,color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
625/625 [==============================] - 36s 58ms/step - loss: 0.8093 - acc: 0.5781 - val_loss: 0.6732 - val_acc: 0.6105
Epoch 2/5
625/625 [==============================] - 30s 48ms/step - loss: 0.6543 - acc: 0.6271 - val_loss: 0.6940 - val_acc: 0.5858
Epoch 3/5
625/625 [==============================] - 30s 48ms/step - loss: 0.6428 - acc: 0.6391 - val_loss: 0.6212 - val_acc: 0.6516
Epoch 4/5
625/625 [==============================] - 30s 48ms/step - loss: 0.6425 - acc: 0.6365 - val_loss: 0.6811 - val_acc: 0.5611
Epoch 5/5
625/625 [==============================] - 30s 48ms/step - loss: 0.6361 - acc: 0.6465 - val_loss: 0.6350 - val_acc: 0.6525
time:155.75570154190063


Fine-Tune

In [10]:
conv_base.trainable=True

In [11]:
train = False
for layer in train_model.layers[1].layers:
    if 'conv5' in layer.name:
        train = True
    layer.trainable = train

In [12]:
train_model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(lr=1e-4))

In [13]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=2,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/2
625/625 [==============================] - 48s 77ms/step - loss: 0.5952 - val_loss: 0.5997
Epoch 2/2
625/625 [==============================] - 39s 63ms/step - loss: 0.5756 - val_loss: 0.5977
time:91.52348232269287


In [14]:
train_model.save('Models/NewWhale.h5')